In [37]:
import pandas as pd

In [38]:
from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer, Text, text

In [39]:
# Load
df = pd.read_csv('netflix_titles.csv')

In [56]:
df.columns

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')

In [57]:
df.head(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [43]:
engine = create_engine("mysql+pymysql://.../netflixdb?charset=utf8mb4", future=True)

In [44]:
# define schema
metadata = MetaData()
netflix_raw = Table(
    "netflix_raw", metadata,
    Column("show_id", String(20)),
    Column("type", String(20)),
    Column("title", String(255)),
    Column("director", String(255)),
    Column("cast", Text),
    Column("country", String(255)),
    Column("date_added", String(50)),
    Column("release_year", Integer),
    Column("rating", String(10)),
    Column("duration", String(20)),
    Column("listed_in", String(255)),
    Column("description", Text),
)

In [53]:
# create table
with engine.begin() as conn:
    metadata.drop_all(conn, tables=[netflix_raw])
    metadata.create_all(conn, tables=[netflix_raw])

# bulk insert
records = df.where(df.notnull(), None).to_dict(orient="records")
chunk = 5000
for i in range(0, len(records), chunk):
    with engine.begin() as conn:
        conn.execute(netflix_raw.insert(), records[i:i+chunk])

In [54]:
# verify (pure SQLAlchemy)
with engine.connect() as c:
    print("rows_loaded:", c.execute(text("SELECT COUNT(*) FROM netflix_raw")).scalar())

rows_loaded: 8807
